In [13]:
from seglearn.feature_functions import base_features, all_features, mean, var, std, skew
from seglearn.transform import SegFeatures, Segment
from seglearn.pipe import SegPipe
from seglearn.datasets import load_watch
from seglearn.util import make_ts_data

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
import matplotlib.pyplot as plt

In [14]:
data = load_watch()
print(data.keys())

fts = all_features()

feed = Pipeline([('segment', Segment()),('features', SegFeatures(fts))])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
y = data['y']
X = make_ts_data(data['X'])

pipe.fit(X, y)
print(pipe.f_labels)
ye, yp = pipe.predict(X, data['y'])
print(pipe.score(X, data['y']))

dict_keys(['y', 'side', 'X', 'y_labels', 'subject', 'X_labels'])
['std_0', 'std_1', 'std_2', 'std_3', 'std_4', 'std_5', 'min_0', 'min_1', 'min_2', 'min_3', 'min_4', 'min_5', 'mse_0', 'mse_1', 'mse_2', 'mse_3', 'mse_4', 'mse_5', 'mean_0', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'mnx_0', 'mnx_1', 'mnx_2', 'mnx_3', 'mnx_4', 'mnx_5', 'skew_0', 'skew_1', 'skew_2', 'skew_3', 'skew_4', 'skew_5', 'corr_0', 'corr_1', 'corr_2', 'corr_3', 'corr_4', 'corr_5', 'corr_6', 'corr_7', 'corr_8', 'corr_9', 'corr_10', 'corr_11', 'corr_12', 'corr_13', 'corr_14', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'hist4_0', 'hist4_1', 'hist4_2', 'hist4_3', 'hist4_4', 'hist4_5', 'hist4_6', 'hist4_7', 'hist4_8', 'hist4_9', 'hist4_10', 'hist4_11', 'hist4_12', 'hist4_13', 'hist4_14', 'hist4_15', 'hist4_16', 'hist4_17', 'hist4_18', 'hist4_19', 'hist4_20', 'hist4_21', 'hist4_22', 'hist4_23', 'max_0', 'max_1', 'max_2', 'max_3', 'max_4', 'max_5', 'kurt_0', 'kurt_1', 'kurt_2', 'kurt_3', 'kurt_4', 'kurt_5

In [15]:
from seglearn.transform import SegFeatures, Segment
from seglearn.pipe import SegPipe
from seglearn.datasets import load_watch
from seglearn.util import check_ts_data, ts_stats
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import numpy as np
data = load_watch()

y = data['y']
X = make_ts_data(data['X'])

check_ts_data(X)
print(ts_stats(X, y, fs=50))

fts = {'mean': mean, 'var': var, 'std': std, 'skew': skew}
feed = Pipeline([('segment', Segment()),('features', SegFeatures(fts))])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)


pipe.fit(X, y)
print(pipe.f_labels)
print(pipe.score(X, y))
      

{'total': {'Series_Time_Mean': 34.87171428571428, 'n_series': 140, 'n_static_vars': 0, 'n_TS_vars': 6, 'Total_time': 4882.04, 'Series_Time_Range': (18.94, 52.36), 'n_classes': 7, 'Series_Time_Std': 8.757850351285423}, 'by_class': {'Class_labels': array([0, 1, 2, 3, 4, 5, 6]), 'n_series': array([20, 20, 20, 20, 20, 20, 20]), 'Series_Time_Min': array([21.28, 22.48, 22.3 , 19.4 , 21.3 , 19.26, 18.94]), 'Series_Time_Mean': array([26.622, 39.905, 40.498, 37.395, 37.604, 30.578, 31.5  ]), 'Total_Time': array([532.44, 798.1 , 809.96, 747.9 , 752.08, 611.56, 630.  ]), 'Series_Time_Max': array([31.1 , 49.1 , 50.84, 52.36, 49.24, 45.12, 38.66]), 'Series_Time_Std': array([2.72084472, 8.0555543 , 8.77209872, 9.08521189, 7.98387775,
       6.09925209, 5.95709997])}}
['skew_0', 'skew_1', 'skew_2', 'skew_3', 'skew_4', 'skew_5', 'mean_0', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'std_0', 'std_1', 'std_2', 'std_3', 'std_4', 'std_5']
0.99871

In [16]:
# univariate time series test and univariate feature test

def uv(X):
    X_new = [X[i][:,0] for i in np.arange(len(X))]
    return X_new
data = load_watch()

Xuv = uv(data['X'])
Xuv = make_ts_data(Xuv)
y = data['y']

check_ts_data(Xuv)
print(ts_stats(Xuv, y))

features = all_features()
del features['corr']
features = {'mean':mean}

feed = Pipeline([('segment', Segment()),('features', SegFeatures(features=features))])
est = RandomForestClassifier()

pipe = SegPipe(feed, est)
pipe.fit(Xuv, y)
print(pipe.f_labels)
print(pipe.score(Xuv, y))


{'total': {'Series_Time_Mean': 1743.5857142857142, 'n_series': 140, 'n_static_vars': 0, 'n_TS_vars': 1, 'Total_time': 244102.0, 'Series_Time_Range': (947.0, 2618.0), 'n_classes': 7, 'Series_Time_Std': 437.89251756427115}, 'by_class': {'Class_labels': array([0, 1, 2, 3, 4, 5, 6]), 'n_series': array([20, 20, 20, 20, 20, 20, 20]), 'Series_Time_Min': array([1064., 1124., 1115.,  970., 1065.,  963.,  947.]), 'Series_Time_Mean': array([1331.1 , 1995.25, 2024.9 , 1869.75, 1880.2 , 1528.9 , 1575.  ]), 'Total_Time': array([26622., 39905., 40498., 37395., 37604., 30578., 31500.]), 'Series_Time_Max': array([1555., 2455., 2542., 2618., 2462., 2256., 1933.]), 'Series_Time_Std': array([136.04223609, 402.77771475, 438.60493613, 454.26059426,
       399.19388773, 304.96260426, 297.85499828])}}
['mean_0']
0.9424844986102202


In [17]:
from seglearn.split import TemporalKFold
from seglearn.datasets import load_watch
data = load_watch()
y = data['y']
X = make_ts_data(data['X'])

splitter = TemporalKFold(n_splits=4)
X, y, cv = splitter.split(X, y)

feed = Pipeline([('segment', Segment()),('features', SegFeatures())])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
cross_validate(pipe, X, y, cv = cv)

/home/david/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.79819107, 0.77020335, 0.7626636 , 0.76601601]),
 'score_time': array([0.13529086, 0.13372278, 0.13658762, 0.13371086]),
 'test_score': array([0.89791873, 0.93557978, 0.92864222, 0.9345887 ]),
 'train_score': array([0.9983482 , 0.99900892, 0.99900892, 0.99933928])}

In [10]:
from seglearn.split import TemporalKFold
from seglearn.datasets import load_watch
Xt = data['X']
Xs = np.column_stack([data['side'],data['subject']])
X = make_ts_data(Xt, Xs)
y = data['y']

check_ts_data(X)
print(ts_stats(X, y))

feed = Pipeline([('segment', Segment()),('features', SegFeatures())])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
pipe.fit(X, y)
print(pipe.f_labels)
print(pipe.score(X, y))

splitter = TemporalKFold(n_splits=4)
X, y, cv = splitter.split(X, y)

feed = Pipeline([('segment', Segment()),('features', SegFeatures())])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
cross_validate(pipe, X, y, cv = cv)



{'total': {'Series_Time_Mean': 1743.5857142857142, 'n_series': 140, 'n_static_vars': 2, 'n_TS_vars': 6, 'Total_time': 244102.0, 'Series_Time_Range': (947.0, 2618.0), 'n_classes': 7, 'Series_Time_Std': 437.89251756427115}, 'by_class': {'Class_labels': array([0, 1, 2, 3, 4, 5, 6]), 'n_series': array([20, 20, 20, 20, 20, 20, 20]), 'Series_Time_Min': array([1064., 1124., 1115.,  970., 1065.,  963.,  947.]), 'Series_Time_Mean': array([1331.1 , 1995.25, 2024.9 , 1869.75, 1880.2 , 1528.9 , 1575.  ]), 'Total_Time': array([26622., 39905., 40498., 37395., 37604., 30578., 31500.]), 'Series_Time_Max': array([1555., 2455., 2542., 2618., 2462., 2256., 1933.]), 'Series_Time_Std': array([136.04223609, 402.77771475, 438.60493613, 454.26059426,
       399.19388773, 304.96260426, 297.85499828])}}
['mnx_0', 'mnx_1', 'mnx_2', 'mnx_3', 'mnx_4', 'mnx_5', 'std_0', 'std_1', 'std_2', 'std_3', 'std_4', 'std_5', 'mean_0', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4

/home/david/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.78452849, 0.76247597, 0.76302314, 0.76041365]),
 'score_time': array([0.13592434, 0.13475561, 0.13512301, 0.135396  ]),
 'test_score': array([0.89890981, 0.95044599, 0.92765114, 0.93657086]),
 'train_score': array([0.99867856, 0.9983482 , 0.9983482 , 0.99735712])}

In [18]:
from sklearn.utils.estimator_checks import check_estimator

# check_estimator(SegFeatures)
check_estimator(Segment)

IndexError: invalid index to scalar variable.

In [20]:
from seglearn.transform import SegFeatures, Segment
from seglearn.feature_functions import mean, var, std, skew
from seglearn.pipe import SegPipe
from seglearn.datasets import load_watch    
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
data = load_watch()
fts = {'mean': mean, 'var': var, 'std': std, 'skew': skew}
feed = Pipeline([('segment', Segment()),('features', SegFeatures(fts))])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
pipe.fit(data['X'], data['y'])
print(pipe.score(data['X'], data['y']))

0.9982895018174043
